In [2]:
import numpy as np

In [318]:
class ButterflyOptimization():
    def __init__(self):
        self.objective_function=None
        
    def compile_algorithm(self,obj,dimension,lb,ub,c=0.5,p=0.8,population_size=100,max_iter=100,optimization="maximize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.c = c
        self.p = p
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    ## Initialize the populations of butterfly randomly
    def __initialize_population(self):
        return self.lb+np.random.rand(self.population_size,self.dimension)*(self.ub-self.lb)
    
    
    ## Calculate the fragrance using the formula f=c*I^2
    def __calculate_fragrance(self,I,a):
        return self.c*np.power(I,a)
    
    ## This function will find the best butterfly in a specific iteration
    def __find_best_butterfly(self,populations,I):
        if self.optimization == "maximize":
            return np.array(populations[np.argmax(I)]),np.max(I)
        elif self.optimization == "minimize":
            return np.array(populations[np.argmin(I)]),np.min(I)
        else:
            raise Exception(f"Unknown Optimization Criteria: {self.optimization}")
    
    ## This function will update the global best butterfly
    def __update_global_best_butterfly(self,global_best,current_best):
        if global_best == None:
            return current_best
        
        if self.optimization == "maximize":
            return current_best if current_best[1]>=global_best[1] else global_best
        elif self.optimization == "minimize":
            return current_best if current_best[1]<=global_best[1] else global_best
    
    ## This is the population update function:
    ## if r < p:
    ## next_population = current_population + (r^2*best_butterfly-current_population)*fragrance
    ## else:
    ## next_population_i = current_population_i + (r^2*current_population_j-current_population_k)*fragrance
    
    def __update_population(self,current_population,best_butterfly,F):
        r = np.random.rand(self.population_size,1)
        j = current_population.copy()
        k = current_population.copy()
        
        np.random.shuffle(j)
        np.random.shuffle(k)
        
        next_population_rlp = current_population + (r**2*best_butterfly-current_population)*np.expand_dims(F,axis=1)
        next_population_rgp = current_population + (r**2*j-k) * np.expand_dims(F,axis=1)
        r = r < self.p
        next_population = r*next_population_rlp+(1-r)*next_population_rgp
        return np.clip(next_population,lb,ub)
    
    def calculate_optimal_value(self):
        if self.objective_function == None:
            raise Exception("You should compile first.")
        global_best = None
        population = self.__initialize_population()
        a = np.random.rand()
        
        for i in range(self.max_iter):
            ## Calculate the value of stimulus intensity I
            I = np.array([self.objective_function(j) for j in population])
            
            ## Calculate the frangrance
            F = self.__calculate_fragrance(I,a)
            
            ## Get the best index of best butterfly from populations
            best_butterfly=self.__find_best_butterfly(population,I)
            
            global_best = self.__update_global_best_butterfly(global_best,best_butterfly)
            
            ## Update the populations 
            populations = self.__update_population(population,global_best[0],F)
            
            a = np.random.rand()
            self.c += 0.025/(self.c*self.max_iter)
        return global_best
    

# Testing the Algorithm with objective function

In [339]:
## Defining a simple objective function
def obj(X):
    return X[0]+X[1]-X[0]/X[1]

lb = np.array([-100,-100])
ub = np.array([100,100])
dim = 2

In [340]:
boa = ButterflyOptimization()

In [341]:
boa.compile_algorithm(obj,dim,lb,ub,c=0.001,population_size=1000,max_iter=1000,optimization="maximize")

In [342]:
print(boa.calculate_optimal_value())

<ipython-input-318-9d4193de9c4a>:23: RuntimeWarning: invalid value encountered in power
  return self.c*np.power(I,a)


(array([68.78174606, -0.31425986]), 287.33650316716495)
